In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Wczytanie danych
df = pd.read_csv("bias_audit_results_full.csv")

# Ustawienia estetyczne
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 8]

# --- WYKRES 1: Średnie wyniki ze względu na płeć (Porównanie modeli) ---
def plot_gender_bias(df):
    # Przekształcamy dane do formatu "long", żeby łatwiej rysować w seaborn
    score_cols = [c for c in df.columns if 'score' in c]
    melted_df = df.melt(id_vars=['gender'], value_vars=score_cols, 
                        var_name='Model_Role', value_name='Score')
    
    plt.figure()
    sns.barplot(data=melted_df, x='gender', y='Score', hue='Model_Role')
    plt.title("Średnie oceny kandydatów ze względu na płeć i rolę")
    plt.ylim(0, 100)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("1_gender_bias.png")
    print("Zapisano wykres 1: Płeć")

# --- WYKRES 2: Wariancja wewnątrz grup (Dowód na Bias) ---
# Skoro w każdej grupie (G1, G2...) wszyscy mają TE SAME SKILLE, 
# to różnica w punktach (max - min) to czysty bias.
def plot_group_variance(df):
    plt.figure()
    # Obliczamy rozstęp ocen w każdej grupie dla konkretnej roli (np. Data Scientist Llama)
    df['llama_ds_range'] = df.groupby('group_id')['Quantitative_Analyst_llama_score'].transform(lambda x: x.max() - x.min())
    
    sns.boxplot(data=df, x='group_id', y='Quantitative_Analyst_llama_score')
    plt.title("Rozrzut ocen w grupach o IDENTYCZNYCH kwalifikacjach (Model: Llama, Rola: Quant Analyst)")
    plt.ylabel("Ocena (0-100)")
    plt.xlabel("ID Grupy Kompetencyjnej")
    plt.savefig("2_group_variance.png")
    print("Zapisano wykres 2: Wariancja grupowa")

# --- WYKRES 3: XAI - Jakie wagi model przypisuje cechom wrażliwym? ---
def plot_feature_weights(df):
    # Wybieramy kolumny z wagami (w_gender, w_age itd.) dla jednego modelu i roli
    weight_cols = [c for c in df.columns if 'Quantitative_Analyst_llama_w_' in c]
    melted_weights = df.melt(value_vars=weight_cols, var_name='Feature', value_name='Weight')
    
    # Skracamy nazwy cech dla czytelności
    melted_weights['Feature'] = melted_weights['Feature'].str.replace('Quantitative_Analyst_llama_w_', '')
    
    plt.figure()
    sns.stripplot(data=melted_weights, x='Feature', y='Weight', jitter=True, alpha=0.5)
    plt.title("Wagi przypisane cechom przez model Llama (XAI Analysis)")
    plt.ylabel("Waga procentowa (%)")
    plt.savefig("3_feature_importance_xai.png")
    print("Zapisano wykres 3: Analiza wag XAI")

# --- WYKRES 4: Mapa ciepła - Narodowość vs Uczelnia ---
def plot_heatmap(df):
    plt.figure(figsize=(14, 10))
    pivot_table = df.pivot_table(
        values='Quantitative_Analyst_llama_score', 
        index='nationality', 
        columns='university', 
        aggfunc='mean'
    )
    sns.heatmap(pivot_table, annot=True, cmap="YlGnBu")
    plt.title("Mapa ciepła: Wpływ pochodzenia i uczelni na ocenę (Llama)")
    plt.tight_layout()
    plt.savefig("4_heatmap_bias.png")
    print("Zapisano wykres 4: Heatmapa")

# Uruchomienie analizy
if __name__ == "__main__":
    try:
        plot_gender_bias(df)
        plot_group_variance(df)
        plot_feature_weights(df)
        plot_heatmap(df)
        print("\nWszystkie analizy gotowe! Sprawdź pliki .png w folderze projektu.")
    except Exception as e:
        print(f"Błąd podczas generowania wykresów: {e}")